### 데이터 값 찾기

* 문서 URL 목록 가져오기

In [11]:
import pandas as pd

# 사업보고서 제무제표 주석 문서 URL 목록
file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/fs_notes_urls/fs_notes_url_all.csv"
fs_notes_all = pd.read_csv(file_path, dtype=object)
# 회사코드 정렬
fs_notes_all = fs_notes_all.sort_values(by=['corp_code'], ascending=True)
# 회사코드 목록
list_all_codes = fs_notes_all['corp_code'].unique()
len(list_all_codes)

342

* 이미 데이터 값을 찾은 것은 대상에서 제외

In [12]:
import os.path

# 이미 데이터 값을 찾은 회사코드 목록
file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/items/items_all.csv"
 
if os.path.exists(file_path):
    try:
        items_all = pd.read_csv(file_path, dtype=object)
        list_own_codes = items_all['corp_code'].unique()
    except pd.errors.EmptyDataError:
        list_own_codes = []    
else:
    list_own_codes = []
    
len(list_own_codes)

342

In [13]:
# 리스트에서 중복 제거 (이미 corp info가 있는 경우 대상에서 제외)
list_target_codes = list(set(list_all_codes) - set(list_own_codes))
len(list_target_codes)

0

* 리스트 분할

In [14]:
list_cnt = len(list_target_codes)
n = 100
list_of_lists = [list_target_codes[i * n:(i + 1) * n] for i in range((list_cnt + n - 1) // n )] 
len(list_of_lists)

0

* 항목과 값 찾기: 확정급여채무의 현재가치, 사외적립자산의 공정가치

In [10]:
import traceback
from time import sleep
from tqdm import tqdm

import sys
module_path = "D:\PythonProject\data-gatherer\dart_fs_notes"
sys.path.append(module_path)
import myutil, mydart

# 파일 저장 위치
path_dir = "D:/PythonProject/data-gatherer/dart_fs_notes/items/"

for corp_codes in list_of_lists:
    for corp_code in tqdm(corp_codes):
        str_expr = "corp_code == @corp_code"
        df_doc = fs_notes_all.query(str_expr)
        doc_count = df_doc.shape[0]
        data_list = []
        for i, doc in df_doc.iterrows():
            sleep(1.0)
            try:
                val_a = val_b = 0.0
                unit = None
                doc_result = doc['doc_result']
                if doc_result == 'No Financial Statement Notes':
                    data_list.append({'corp_code' : doc['corp_code'], 'stock_code' : doc['stock_code'], 'corp_cls' : doc['corp_cls'], 'corp_name' : doc['corp_name'], 
                                    'rpt_num' : doc['rpt_num'], 'rpt_name' : doc['rpt_name'], 
                                    'doc_title' : doc['doc_title'], 'doc_result' : doc_result, 'unit' : unit, 
                                    'val_a' : None, 'val_b' : None, 'item_result' : None})
                else:
                    # 재무제표 주석 문서에서 사용된 금액 단위 찾기 (확정급여채무)
                    unit = mydart.find_unit(doc['doc_url'])
                    # 당기말의 "확정급여채무"와 "사외적립자산" 금액 찾기
                    val_a, val_b, item_result = mydart.find_item(doc['doc_url'])      
                    data_list.append({'corp_code' : doc['corp_code'], 'stock_code' : doc['stock_code'], 'corp_cls' : doc['corp_cls'], 'corp_name' : doc['corp_name'], 
                                    'rpt_num' : doc['rpt_num'], 'rpt_name' : doc['rpt_name'], 
                                    'doc_title' : doc['doc_title'], 'doc_result' : doc_result, 'unit' : unit, 
                                    'val_a' : val_a, 'val_b' : val_b, 'item_result' : item_result})
            except ValueError as err:
                print(doc['corp_code'], doc['corp_name'], doc['rpt_num'], doc['rpt_name'], doc['rpt_url'], doc['doc_title'], doc['doc_url'])
                print(err)
                break
            except Exception as e:
                print(doc['corp_code'], doc['corp_name'], doc['rpt_num'], doc['rpt_name'], doc['rpt_url'], doc['doc_title'], doc['doc_url'])
                err_msg = traceback.format_exc()
                print(err_msg)
                break
        # 결과를 파일로 저장
        doc_value_count = len(data_list)
        if doc_value_count == 0:
            break
        elif doc_value_count == doc_count:        
            df_items = pd.DataFrame(data_list)
            file_name = corp_code + '_item_' + myutil.now_dt_str() + '.csv'
            df_items.to_csv(path_dir + file_name, index=False)
        else:
            continue

100%|██████████| 26/26 [07:01<00:00, 16.23s/it]
